<a href="https://colab.research.google.com/github/Orneyfish/Orney/blob/master/Entity_extraction_from_JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bionlp13cg_md-0.2.4.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

     |████████████████████████████████| 10.0MB 4.1MB/s 
     |████████████████████████████████| 2.1MB 44.7MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


     |████████████████████████████████| 3.2MB 10.7MB/s 
     |████████████████████████████████| 13.0MB 24.0MB/s 
     |████████████████████████████████| 6.3MB 47.6MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 296kB 35.0MB/s 
  Created wheel for scispacy: filename=scispacy-0.2.4-cp36-none-any.whl size=35204 sha256=2299d8722c61ec5c0f7d9d7d33fe16806d641a98502243417e037f7131effb76
  Stored in directory: /root/.cache/pip/wheels/36/5e/7c/ba8d6b02f5cfa40735e40ccffd29852ca19ebca59393afd5f0
Successfully built scispacy
  Found existing installation: botocore 1.17.2
    Uninstalling botocore-1.17.2:
      Successfully uninstalled botocore-1.17.2
  Found existing installation: rsa 4.6
    Uninstalling rsa-4.6:
      Successfully uninstalled rsa-4.6


     |████████████████████████████████| 70.1MB 54kB/s 
  Created wheel for en-ner-bionlp13cg-md: filename=en_ner_bionlp13cg_md-0.2.4-cp36-none-any.whl size=70542687 sha256=e7f1fcb37d2baef80efa9ab74e5d97943b27cba139b8b4673df29014be3ee193
  Stored in directory: /root/.cache/pip/wheels/ef/2a/a6/014a4ac046ce7e0afeba214dc8ca1ab1bd5cbf05c4b77b28ec
Successfully built en-ner-bionlp13cg-md
     |████████████████████████████████| 17.0MB 243kB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.2.4-cp36-none-any.whl size=17161110 sha256=ad35ceb148261b977a986cf42b82be4f20a73eccc306323b3ccdb91e59a19a61
  Stored in directory: /root/.cache/pip/wheels/34/60/b9/fabd9c3eeba17ed66df745479f2fc502a6702755cb4a9632f2
Successfully built en-core-sci-sm


In [ ]:
import json
import numpy as np
import pandas as pd
import time
import spacy
import re
import string
import en_core_web_sm
import requests
import en_ner_bionlp13cg_md
import scispacy
import time

nlpp = en_ner_bionlp13cg_md.load()
nlp = en_core_web_sm.load()

In [ ]:
def annotate_text(text):
    
    base_url = "http://api.dbpedia-spotlight.org/en/annotate"
    params = {"text": "{}".format(text),"confidence": 0.35}
    headers = {'accept': 'application/json'}

    res = requests.get(base_url, params=params, headers=headers)
    
    if res.status_code != 200:
        raise APIError(res.status_code)
    data = json.loads(res.text)
    return data


text="this is a mango of england having corona"

data=annotate_text(text)
data

{'@confidence': '0.35',
 '@policy': 'whitelist',
 '@sparql': '',
 '@support': '0',
 '@text': 'this is a mango of england having corona',
 '@types': '',
 'Resources': [{'@URI': 'http://dbpedia.org/resource/Mango',
   '@offset': '10',
   '@percentageOfSecondRank': '2.575643419037994E-4',
   '@similarityScore': '0.9995042165409311',
   '@support': '2117',
   '@surfaceForm': 'mango',
   '@types': 'Wikidata:Q756,Wikidata:Q19088,DBpedia:Species,DBpedia:Plant,DBpedia:Eukaryote'},
  {'@URI': 'http://dbpedia.org/resource/Corona',
   '@offset': '34',
   '@percentageOfSecondRank': '0.04105857205573001',
   '@similarityScore': '0.942981628093149',
   '@support': '699',
   '@surfaceForm': 'corona',
   '@types': ''}]}

In [ ]:
##ENTITY EXTRACTION FROM SCISPACY MODEL
def get_spacy_entities(document):
    doc = nlpp(document)
    entity = [X.text for X in doc.ents]
    return entity

In [ ]:
##ENTITY EXTRACTION FROM DBPEDIA
class APIError(Exception):
    def __init__(self, status):
        self.status = status
    def __str__(self):
        return "APIError: status={}".format(self.status)

def annotate_text(text):
    
    base_url = "http://api.dbpedia-spotlight.org/en/annotate"
    params = {"text": "{}".format(text),"confidence": 0.35}
    headers = {'accept': 'application/json'}
    res = requests.get(base_url, params=params, headers=headers)
    if res.status_code != 200:
        raise APIError(res.status_code)
    data = json.loads(res.text)
    return data


def dbp(text):
    data = annotate_text(text)
    entities=[]
    if ("Resources" in data):
        resources = data["Resources"]
        entities = list()
        for each in resources:
            entity = each["@surfaceForm"]
            if entity not in entities:
                entities.append(entity)
    return entities


In [ ]:
##ENTITY EXTRACTION OF (NOUN + PRONOUN - COMMON WORDS) FROM SPACY
w="/content/wordslist.txt"
file = open(w,"r")
f = file.readlines()
word = []
for w in f:
    word.append(w.replace("\n", ""))

word

FileNotFoundError: ignored

In [ ]:
max_dbp_size = 60000

def splitting(text):
    words = text.split()
    subs = []
    n = 50
    for i in range(0, len(words), n):
        subs.append(" ".join(words[i:i+n]))
    return subs

In [ ]:


filelist=[]

import os
for dirname, _, filenames in os.walk('/content/Inputs'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        filelist.append(os.path.join(dirname, filename))

In [ ]:
/
triples=[]
for item1 in filelist:
     with open(item1,"r") as object:
        data = object.read()
        obj = json.loads(data)
        paperid = obj['paper_id']
        title = obj['metadata']['title']
        authors = obj['metadata']['authors']
        abstract = obj['abstract']
        bib = obj['bib_entries']
        body = obj['body_text']
        triples1=[]
        
        titlesp = get_spacy_entities(title)
        for item in titlesp:
            l=[item, paperid, 'has_title', 'SciSpacy']
            triples.append(l)
        title = nlp(title)
        for token in title:
                if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
                        if(len(token.text) > 2):
                                triples1.append(token.text)
        triples1 = list(set(triples1) - set(word))
        for item in triples1:
                l=[item, paperid, 'has_title']
                triples.append(l)
        for item in authors:
                name = item['first'] + " " +  item['last']
                l = [name, paperid, 'has_author']
                triples.append(l)
        abss=""
        for item in abstract:
              abss = abss + item['text'] + " "
              
              if len(abss) >= max_dbp_size:
                  x_list = splitting(abss)
              else:
                   x_list = [abss]
                
              for x_small in x_list:
                    abssd = dbp(x_small)
                    

                    for item in abssd:
                        l=[item, paperid, 'has_abstract', 'DBPedia']
                        triples.append(l)
        abspacy = get_spacy_entities(abss)
        for item in abspacy:
            l=[item, paperid, 'has_abstract', 'SciSpacy']
            triples.append(l)
        abstract = re.sub(r" \d+", "", abss)
        abstract = re.sub(r"[^A-Za-z0-9 -]+", "",abstract)
        abstract = nlp(abstract)
        l1=[]
        abs1=abstract.ents
        for token in abstract:
                if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
                        l1.append(token.text)
        l2=[]
        for token in abs1:
                l2.append(token.text)
        l3 = list(set(l1) | set(l2))
        l3 = list(set(l3) - set(word))
        for item in l3:
                if(len(item) > 2):
                        l=[item, paperid, 'has_abstract']
                        triples.append(l)
       

        btext=""
        bod=[]
        for item in body:
                btext =item['text']
                if len(btext) >= max_dbp_size:
                    x_list = splitting(btext)
                else:
                   x_list = [btext]
                for x_small in x_list:
                    btextdb = dbp(x_small)
                    time.sleep(1)


                    for item in btextdb:
                       l=[item, paperid, 'has_body', 'DBPedia']
                       triples.append(l)
                       
                btextsp = get_spacy_entities(btext)
                for item in btextsp:
                    l=[item, paperid, 'has_body', 'SciSpacy']
                    triples.append(l)
                keyphrases=[]
               # try:
               #         
               #         extractor.load_document(input=btext, language='en')
               #         extractor.candidate_selection()
               #         extractor.candidate_weighting()
               #         keyphrases = extractor.get_n_best(n=5)
               # except ValueError:
               #         keyphrases=[]
              #          btext = re.sub(r" \d+", "", btext)
               #         btext = re.sub(r"[^A-Za-z0-9 -]+", "",btext)

                btext=nlp(btext)
                l1=[]
                b1 = btext.ents
                for token in btext:
                  if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
                     l1.append(token.text)
                l2=[]
                for token in b1:
                   l2.append(token.text)
                l3 = list(set(l1) | set(l2))
                l3 = list(set(l3) - set(word))
                for item2 in l3:
                   x=[item2]
                   keyphrases.append(x)
                for item in keyphrases:
                        
                        bod.append(item[0])
        bod = list(set(bod) - set(word))
        for item in bod:
                l=[item, paperid, 'has_body']
                triples.append(l)

res=pd.DataFrame(triples)
res.to_csv("results.csv",index='False')

In [ ]:
seen = set()
tripless=[]

count = 0
for item in triples:
        t = tuple(item)
        if t not in seen:
                tripless.append(item)
                seen.add(t)

tripless=pd.DataFrame(tripless)
tripless.to_csv("results_duplicate_refined.csv",index='False')

In [ ]:
from google.colab import files
files.download("results_duplicate_refined.csv")

In [ ]:
from google.colab import files
files.download("results.csv")

In [ ]:
!pip install Owlready2

     |████████████████████████████████| 20.8MB 1.4MB/s 
  Created wheel for Owlready2: filename=Owlready2-0.24-cp36-cp36m-linux_x86_64.whl size=20262985 sha256=d6aa6eab48cd25876641ecaa7f0ce0dda5948663186f6c6bd49026da72e863eb
  Stored in directory: /root/.cache/pip/wheels/66/87/0b/7a97bd19c9dc817b279aadb5dd920ce3a77050e2ab5a413f43
Successfully built Owlready2


In [ ]:
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *
default_world.set_backend(filename = "pym.sqlite3")
import_umls("umls-2020AA-metathesaurus.zip", terminologies = ["ICD10", "SNOMEDCT_US", "CUI"])
default_world.save()

In [ ]:
import urllib3
from urllib.request import urlopen